In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pyspark import SparkContext

try:
    sc = SparkContext('local', 'my_context')
except ValueError:
    print('Spark_Context already exists!')

from pyspark.sql import SparkSession
try:
    spark = SparkSession.builder.appName('feature_engineering').getOrCreate()
except ValueError:
    print('Spark_Session already exists!')

### Load Data

#### Pandas

In [2]:
%time
pandas_dataframe = pd.read_csv("titanic_train.csv")
pandas_dataframe.head()

Wall time: 0 ns


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### PySpark

In [4]:
%time
spark_dataframe = spark.read.csv("titanic_train.csv", header=True, inferSchema=True)
print(spark_dataframe.show(5))

Wall time: 0 ns
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+--

### Summary Statistics

#### Pandas

In [5]:
pandas_dataframe.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### PySpark

In [6]:
spark_dataframe.describe().show(truncate=True)

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

### Correlation

#### Pandas

In [8]:
pandas_dataframe[['Age','Pclass', 'Survived']].corr()

,Age,Pclass,Survived
Age,1.000000,-0.369226,-0.077221
Pclass,-0.369226,1.000000,-0.338481
Survived,-0.077221,-0.338481,1.000000


#### PySpark

In [9]:
from pyspark.sql.functions import corr
spark_dataframe.select(corr("Age","Pclass","Survived")).show()

TypeError: corr() takes 2 positional arguments but 3 were given

Note: PySpark only takes two values to make correlation at a time!!!

In [10]:
from pyspark.sql.functions import corr
spark_dataframe.select(corr("Age","Survived")).show()

+--------------------+
| corr(Age, Survived)|
+--------------------+
|-0.07722109457217759|
+--------------------+



### Aggregation and Grouping

#### Pandas

In [13]:
pandas_dataframe.groupby('Pclass')['Age','Fare'].min()

C:\Users\Joshua Amresh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Age,Fare
Pclass,,
1,0.92,0.0
2,0.67,0.0
3,0.42,0.0


In [14]:
pandas_dataframe.groupby('Pclass')['Age','Fare'].max()

C:\Users\Joshua Amresh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Age,Fare
Pclass,,
1,80.0,512.3292
2,70.0,73.5000
3,74.0,69.5500


#### PySpark

In [15]:
from pyspark.sql import functions as F
spark_dataframe.groupBy("Pclass").agg(F.max("Age"),F.max("Fare")).show()

+------+--------+---------+
|Pclass|max(Age)|max(Fare)|
+------+--------+---------+
|     1|    80.0| 512.3292|
|     3|    74.0|    69.55|
|     2|    70.0|     73.5|
+------+--------+---------+



### Missing Data Imputation

#### Pandas

In [16]:
pandas_dataframe.fillna('NA', inplace = True)
pandas_dataframe.fillna(0, inplace = True)

Pandas DataFrame provides a fillna() function that can fill missing data items with any string or number.

Spark DataFrames provide a similar function, but they only allow a value that matches the data type of the corresponding column.

#### PySpark

In [17]:
spark_dataframe = spark_dataframe.na.fill(0)

### Data Filtering

#### Pandas

In [18]:
pandas_dataframe.loc[pandas_dataframe['Pclass'] == 2, ['Age','Fare']].head()

,Age,Fare
9,14.0,30.0708
15,55.0,16.0000
17,NA,13.0000
20,35.0,26.0000
21,34.0,13.0000


#### PySpark

In [19]:
spark_dataframe.filter("Pclass == 2").select("Age","Fare").show(5)

+----+-------+
| Age|   Fare|
+----+-------+
|14.0|30.0708|
|55.0|   16.0|
| 0.0|   13.0|
|35.0|   26.0|
|34.0|   13.0|
+----+-------+
only showing top 5 rows



## Feature Engineering

### Column Selection

In [20]:
pdf=pandas_dataframe[['Age', 'Fare']]
sdf=spark_dataframe.select('Age', 'Fare')

In [21]:
pdf.head()

,Age,Fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500


In [22]:
sdf.show(5)

+----+-------+
| Age|   Fare|
+----+-------+
|22.0|   7.25|
|38.0|71.2833|
|26.0|  7.925|
|35.0|   53.1|
|35.0|   8.05|
+----+-------+
only showing top 5 rows



### Renaming Columns

In [23]:
pandas_dataframe.rename(columns = {"Age": "Vayasu"}, inplace = True)
spark_dataframe = spark_dataframe.withColumnRenamed("Fare", "Kaasu")

In [24]:
pandas_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Vayasu       891 non-null    object 
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(1), int64(5), object(6)
memory usage: 83.7+ KB


In [25]:
spark_dataframe.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Kaasu: double (nullable = false)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### New Feature Creation

#### Pandas

In [28]:
pandas_dataframe['diff'] = pandas_dataframe['Survived'] - pandas_dataframe['Pclass']

In [29]:
pandas_dataframe.head()

,PassengerId,Survived,Pclass,Name,Sex,Vayasu,SibSp,Parch,Ticket,Fare,Cabin,Embarked,diff
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NA,S,-3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NA,S,-2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NA,S,-3


#### PySpark

In [30]:
from pyspark.sql.functions import col
spark_dataframe = spark_dataframe.withColumn("diff", col("Survived") - col("Pclass"))

In [31]:
spark_dataframe.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|  Kaasu|Cabin|Embarked|diff|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|  -3|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  -2|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|  -3|
+-----------+--------+------+-----------

### Removing Unwanted Features

#### Pandas

In [32]:
pandas_dataframe = pandas_dataframe.drop(['diff'], axis=1)

In [33]:
pandas_dataframe.head()

,PassengerId,Survived,Pclass,Name,Sex,Vayasu,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NA,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NA,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NA,S


#### PySpark

In [34]:
spark_dataframe = spark_dataframe.drop('diff')

In [35]:
spark_dataframe.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|  Kaasu|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

### Feature Scaling

#### Pandas

In [36]:
from sklearn.preprocessing import MinMaxScaler

In [37]:
scaler = MinMaxScaler()

In [39]:
pandas_dataframe['fare_scaled'] = scaler.fit_transform(pandas_dataframe['Fare'].values.reshape(-1,1))

In [40]:
pandas_dataframe.head()

,PassengerId,Survived,Pclass,Name,Sex,Vayasu,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fare_scaled
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NA,S,0.014151
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.139136
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NA,S,0.015469
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.103644
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NA,S,0.015713


#### PySpark

In [41]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

In [42]:
# VectorAssembler Transformation - Converting column to vector type
assembler = VectorAssembler(inputCols=['Kaasu'], outputCol="Kaasu_vec")
scaler = MinMaxScaler(inputCol="Kaasu_vec", outputCol="Kaasu_scaled")
# Pipeline of VectorAssembler and MinMaxScaler
pipeline = Pipeline(stages=[assembler, scaler])
# Fitting pipeline on dataframe
spark_df_scaled = pipeline.fit(spark_dataframe).transform(spark_dataframe).drop("Kaasu_vec")

In [43]:
spark_df_scaled.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|  Kaasu|Cabin|Embarked|        Kaasu_scaled|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|[0.01415105756220...|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|[0.13913573538264...|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|[0.01546856981799...|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|[0.10364429745562...|
|          5|       0|     3|Allen, Mr. Willia..

### One Hot Encoding

#### Pandas

In [44]:
pandas_dataframe_ohe = pd.get_dummies(pandas_dataframe)

In [45]:
pandas_dataframe_ohe.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,fare_scaled,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_NA,Cabin_T,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
0,1,0,3,1,0,7.2500,0.014151,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,2,1,1,1,0,71.2833,0.139136,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,3,1,3,0,0,7.9250,0.015469,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,4,1,1,1,0,53.1000,0.103644,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,3,0,0,8.0500,0.015713,0,0,0,...,0,0,0,0,1,0,0,0,0,1


#### PySpark

In [49]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType

categories = spark_dataframe.select('Sex').distinct().rdd.flatMap(lambda x : x).collect()

categories.sort()

for category in categories:
    function = udf(lambda item: 1 if item == category else 0, IntegerType())
    new_column_name = 'Sex'+'_'+category
    spark_dataframe = spark_dataframe.withColumn(new_column_name, function(col('Sex')))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 66 in stage 30.0 failed 1 times, most recent failure: Lost task 66.0 in stage 30.0 (TID 223) (DESKTOP-55442NH executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.ap

In [50]:
spark_dataframe.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|  Kaasu|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male| 0.0|    0|    0|      